In [11]:
# face recognition part II
#IMPORT
import cv2 as cv
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import pickle
from keras_facenet import FaceNet
#INITIALIZE
facenet = FaceNet()
faces_embeddings = np.load("faces_embeddings_done_4classes.npz")
Y = faces_embeddings['arr_1']
encoder = LabelEncoder()
encoder.fit(Y)
haarcascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
model = pickle.load(open("svm_model_160x160.pkl", 'rb'))

cap = cv.imread("1.jpg")
# WHILE LOOP
name=[]
frame = cap
rgb_img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
faces = haarcascade.detectMultiScale(gray_img, 1.3, 5)
for x,y,w,h in faces:
        img = rgb_img[y:y+h, x:x+w]
        img = cv.resize(img, (160,160)) # 1x160x160x3
        img = np.expand_dims(img,axis=0)
        ypred = facenet.embeddings(img)
        face_name = model.predict(ypred)
        final_name = encoder.inverse_transform(face_name)[0]
        name.append(final_name)

print(name[:][:])


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
[np.str_('Saurabh_Chouhan')]


In [33]:
import cv2 as cv
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from mtcnn import MTCNN
from scipy.spatial.distance import euclidean

# Initialize FaceNet and MTCNN
facenet = FaceNet()
detector = MTCNN()

# Load known face embeddings
faces_embeddings = np.load("faces_embeddings_done_4classes.npz")
X_known = faces_embeddings['arr_0']  # Known face embeddings
Y_known = faces_embeddings['arr_1']  # Corresponding labels

encoder = LabelEncoder()
encoder.fit(Y_known)

# Load trained SVM model
model = pickle.load(open("svm_model_160x160.pkl", 'rb'))

# Read input image
image_path = "2.jpg"
cap = cv.imread(image_path)

if cap is None:
    print("Error: Image not found or incorrect path.")
else:
    recognized_names = []
    rgb_img = cv.cvtColor(cap, cv.COLOR_BGR2RGB)

    # Detect faces using MTCNN
    faces = detector.detect_faces(rgb_img)

    if len(faces) == 0:
        print("No faces detected.")
    else:
        i=0
        for face in faces:
            x, y, w, h = face['box']
            x, y = max(0, x), max(0, y)  # Ensure values are positive

            img = rgb_img[y:y+h, x:x+w]
            img = cv.resize(img, (160, 160))  # Resize for FaceNet
            img = np.expand_dims(img, axis=0)

            # Extract face embeddings
            ypred = facenet.embeddings(img)  # Shape: (1, embedding_size)

            # Check if embeddings are valid
            if np.all(ypred == 0):
                print(f"Invalid embeddings for face at ({x}, {y}). Skipping.")
                continue

            # Predict the face identity
            face_name = model.predict(ypred)
            final_name = encoder.inverse_transform(face_name)[0]

            # Compute distance from known embeddings
            distances = [euclidean(ypred.flatten(), known_emb) for known_emb in X_known]
            min_distance = min(distances)

            # Define threshold (tune this value for better accuracy)
            threshold = 1.0  # Adjust this (lower = stricter)

            # Only accept if the distance is below the threshold
            if min_distance < threshold:
                recognized_names.append(final_name)
                print(f"✔ Recognized: {final_name} (Distance: {min_distance:.2f})")
            else:
                recognized_names.append("unknown" + str(i))
                i=i+1
                print(f"❌ Unknown face at ({x}, {y}) rejected (Distance: {min_distance:.2f})")

        print("Final Recognized Persons:", list(set(recognized_names)))  # Output identified people


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
❌ Unknown face at (953, 739) rejected (Distance: 1.04)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
✔ Recognized: uttam (Distance: 0.60)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
✔ Recognized: Lokendra_Singh (Distance: 0.63)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
✔ Recognized: ashish (Distance: 0.54)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
❌ Unknown face at (42, 42) rejected (Distance: 1.16)
Final Recognized Persons: ['unknown0', 'unknown1', np.str_('Lokendra_Singh'), np.str_('uttam'), np.str_('ashish')]
